importing libraries

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input,LSTM,Dense
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pickle


In [4]:
#initialize all variables 
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()


In [6]:
#read dataset file
with open('try.txt','r',encoding='utf-8') as f:
    rows=f.read().split('\n')

In [7]:
for row in rows[:8]:
    #split input and target by '\t'=tab
    input_text,target_text = row.split('\t')
    #add '\t' at start and '\n' at end of text.
    target_text='\t' + target_text + '\n'
    input_texts.append(input_text.lower())
    target_texts.append(target_text)
    #split character from text and add in respective sets
    input_characters.update(list(input_text.lower()))
    target_characters.update(list(target_text))


In [9]:
#sort input and target characters 
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [10]:
#get the total length of input and target characters
num_en_chars = len(input_characters)
num_dec_chars = len(target_characters)

In [13]:
#get the maximum length of input and target text.
max_input_length = max([len(i) for i in input_texts])
max_target_length = max([len(i) for i in target_texts])


In [14]:
def bagofcharacters(input_texts,target_texts):
  #inintialize encoder , decoder input and target data.
  en_in_data=[] ; dec_in_data=[] ; dec_tr_data=[]
  #padding variable with first character as 1 as rest all 0.
  pad_en=[1]+[0]*(len(input_characters)-1)
  pad_dec=[0]*(len(target_characters)) ; pad_dec[2]=1
  #countvectorizer for one hot encoding as we want to tokenize character so
  #anlyzer is true and None the stopwords action.
  cv=CountVectorizer(binary=True,tokenizer=lambda txt: txt.split(),stop_words=None,analyzer='char')
  for i,(input_t,target_t) in enumerate(zip(input_texts,target_texts)):
    #fit the input characters into the CountVectorizer function
    cv_inp= cv.fit(input_characters)
    
    #transform the input text from the help of CountVectorizer fit.
    #it character present than put 1 and 0 otherwise.
    en_in_data.append(cv_inp.transform(list(input_t)).toarray().tolist())
    cv_tar= cv.fit(target_characters)		
    dec_in_data.append(cv_tar.transform(list(target_t)).toarray().tolist())
    #decoder target will be one timestep ahead because it will not consider 
    #the first character i.e. '\t'.
    dec_tr_data.append(cv_tar.transform(list(target_t)[1:]).toarray().tolist())
    
    #add padding variable if the length of the input or target text is smaller
    #than their respective maximum input or target length. 
    if len(input_t) < max_input_length:
      for _ in range(max_input_length-len(input_t)):
        en_in_data[i].append(pad_en)
    if len(target_t) < max_target_length:
      for _ in range(max_target_length-len(target_t)):
        dec_in_data[i].append(pad_dec)
    if (len(target_t)-1) < max_target_length:
      for _ in range(max_target_length-len(target_t)+1):
        dec_tr_data[i].append(pad_dec)
  
  #convert list to numpy array with data type float32
  en_in_data=np.array(en_in_data,dtype="float32")
  dec_in_data=np.array(dec_in_data,dtype="float32")
  dec_tr_data=np.array(dec_tr_data,dtype="float32")

  return en_in_data,dec_in_data,dec_tr_data

In [15]:
#create input object of total number of encoder characters
en_inputs = Input(shape=(None, num_en_chars))

In [16]:
#create LSTM with the hidden dimension of 256
#return state=True as we don't want output sequence.
encoder = LSTM(256, return_state=True)

In [17]:
#discard encoder output and store hidden and cell state.
en_outputs, state_h, state_c = encoder(en_inputs)
en_states = [state_h, state_c]

In [18]:
#create input object of total number of decoder characters
dec_inputs = Input(shape=(None, num_dec_chars))

In [19]:
#create LSTM with the hidden dimension of 256
#return state and return sequences as we want output sequence.
dec_lstm = LSTM(256, return_sequences=True, return_state=True)

In [20]:
#initialize the decoder model with the states on encoder.
dec_outputs, _, _ = dec_lstm(dec_inputs, initial_state=en_states)

In [21]:
#Output layer with shape of total number of decoder characters 
dec_dense = Dense(num_dec_chars, activation="softmax")
dec_outputs = dec_dense(dec_outputs)

In [22]:
#create Model and store all variables 
model = Model([en_inputs, dec_inputs], dec_outputs)
pickle.dump({'input_characters':input_characters,'target_characters':target_characters,
             'max_input_length':max_input_length,'max_target_length':max_target_length,
             'num_en_chars':num_en_chars,'num_dec_chars':num_dec_chars},open("training_data.pkl","wb"))


In [23]:
#load the data and train the model
en_in_data,dec_in_data,dec_tr_data = bagofcharacters(input_texts,target_texts)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [en_in_data, dec_in_data],
    dec_tr_data,
    batch_size=64,
    epochs=200,
    validation_split=0.2,
)
# Save model
model.save("s2s")

#summary and model plot
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:551: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


Epoch 1/200
1/1 [==============================] - 12s 12s/step - loss: 3.0491 - accuracy: 0.1296 - val_loss: 3.0251 - val_accuracy: 0.1667
Epoch 2/200
1/1 [==============================] - 0s 138ms/step - loss: 3.0098 - accuracy: 0.3333 - val_loss: 3.0003 - val_accuracy: 0.4444
Epoch 3/200
1/1 [==============================] - 0s 138ms/step - loss: 2.9687 - accuracy: 0.4630 - val_loss: 2.9717 - val_accuracy: 0.4444
Epoch 4/200
1/1 [==============================] - 0s 134ms/step - loss: 2.9223 - accuracy: 0.4630 - val_loss: 2.9361 - val_accuracy: 0.4444
Epoch 5/200
1/1 [==============================] - 0s 134ms/step - loss: 2.8656 - accuracy: 0.4444 - val_loss: 2.8889 - val_accuracy: 0.4444
Epoch 6/200
1/1 [==============================] - 0s 146ms/step - loss: 2.7919 - accuracy: 0.4074 - val_loss: 2.8234 - val_accuracy: 0.4444
Epoch 7/200
1/1 [==============================] - 0s 152ms/step - loss: 2.6915 - accuracy: 0.4074 - val_loss: 2.7291 - val_accuracy: 0.4444
Epoch 8/200
1/

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 18)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 21)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 281600      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  284672      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [24]:
from tkinter import *
import pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras.layers import Input,LSTM,Dense

In [25]:
BG_GRAY="#ABB2B9"
BG_COLOR="#000"
TEXT_COLOR="#FFF"
FONT="Melvetica 14"
FONT_BOLD="Melvetica 13 bold"


In [26]:
cv=CountVectorizer(binary=True,tokenizer=lambda txt: txt.split(),stop_words=None,analyzer='char') 

In [27]:
class LangTRans:
    def __init__(self):
        #initialize tkinter window and load the file
        self.window=Tk()
        self.main_window()
        self.datafile()

    def datafile(self):
        #get all datas from datafile and load the model.
        datafile = pickle.load(open("training_data.pkl","rb"))
        self.input_characters = datafile['input_characters']
        self.target_characters = datafile['target_characters']
        self.max_input_length = datafile['max_input_length']
        self.max_target_length = datafile['max_target_length']
        self.num_en_chars = datafile['num_en_chars']
        self.num_dec_chars = datafile['num_dec_chars']
        self.loadmodel()

    #runwindow
    def run(self):
        self.window.mainloop()
    
    def main_window(self):
        #add title to window and configure it
        self.window.title("Language Translator")
        self.window.resizable(width=False,height=False)
        self.window.configure(width=520,height=520,bg=BG_COLOR)
    
        head_label=Label(self.window,bg=BG_COLOR,fg=TEXT_COLOR,text="Welcome to DataFlair",font=FONT_BOLD,pady=10)
        head_label.place(relwidth=1)
        line = Label(self.window,width=450,bg=BG_COLOR)
        line.place(relwidth=1,rely=0.07,relheight=0.012)

        #create text widget where input and output will be displayed
        self.text_widget=Text(self.window,width=20,height=2,bg="#fff",fg="#000",font=FONT,padx=5,pady=5)
        self.text_widget.place(relheight=0.745,relwidth=1,rely=0.08)
        self.text_widget.configure(cursor="arrow",state=DISABLED)

        #create scrollbar
        scrollbar=Scrollbar(self.text_widget)
        scrollbar.place(relheight=1,relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)

        #create bottom label where text widget will placed
        bottom_label=Label(self.window,bg=BG_GRAY,height=80)
        bottom_label.place(relwidth=1,rely=0.825)
        #this is for user to put english text
        self.msg_entry=Entry(bottom_label,bg="#2C3E50",fg=TEXT_COLOR,font=FONT)
        self.msg_entry.place(relwidth=0.788,relheight=0.06,rely=0.008,relx=0.008)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>",self.on_enter)
        #send button which will call on_enter function to send the text
        send_button=Button(bottom_label,text="Send",font=FONT_BOLD,width=8,bg="#fff",command=lambda: self.on_enter(None))        
        send_button.place(relx=0.80,rely=0.008,relheight=0.06,relwidth=0.20)

    def loadmodel(self):
        #Inference model
        #load the model
        model = models.load_model("s2s")
        #construct encoder model from the output of second layer
        #discard the encoder output and store only states.
        enc_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
        #add input object and state from the layer.
        self.en_model = Model(model.input[0], [state_h_enc, state_c_enc])

        #create Input object for hidden and cell state for decoder
        #shape of layer with hidden or latent dimension
        dec_state_input_h = Input(shape=(256,), name="input_3")
        dec_state_input_c = Input(shape=(256,), name="input_4")
        dec_states_inputs = [dec_state_input_h, dec_state_input_c]

        #add input from the encoder output and initialize with 
        #states.
        dec_lstm = model.layers[3]
        dec_outputs, state_h_dec, state_c_dec = dec_lstm(
            model.input[1], initial_state=dec_states_inputs
        )
        dec_states = [state_h_dec, state_c_dec]
        dec_dense = model.layers[4]
        dec_outputs = dec_dense(dec_outputs)
        #create Model with the input of decoder state input and encoder input
        #and decoder output with the decoder states.
        self.dec_model = Model(
            [model.input[1]] + dec_states_inputs, [dec_outputs] + dec_states
        )
        
    def decode_sequence(self,input_seq):
        #create dict object to get character from the index.
        reverse_target_char_index = dict(enumerate(self.target_characters))
        #get the states from the user input sequence
        states_value = self.en_model.predict(input_seq)

        #fit target characters and 
        #initialize every first character to be 1 which is '\t'.
        #Generate empty target sequence of length 1.
        co=cv.fit(self.target_characters) 
        target_seq=np.array([co.transform(list("\t")).toarray().tolist()],dtype="float32")

        #if the iteration reaches the end of text than it will be stop the it
        stop_condition = False
        #append every predicted character in decoded sentence
        decoded_sentence = ""
        while not stop_condition:
            #get predicted output and discard hidden and cell state.
            output_chars, h, c = self.dec_model.predict([target_seq] + states_value)

            #get the index and from dictionary get character from it.
            char_index = np.argmax(output_chars[0, -1, :])
            text_char = reverse_target_char_index[char_index]
            decoded_sentence += text_char

            # Exit condition: either hit max length
            # or find stop character.
            if text_char == "\n" or len(decoded_sentence) > self.max_target_length:
                stop_condition = True
            #update target sequence to the current character index.
            target_seq = np.zeros((1, 1, self.num_dec_chars))
            target_seq[0, 0, char_index] = 1.0
            states_value = [h, c]
        #return the decoded sentence
        return decoded_sentence

    def on_enter(self,event):
        #get user query and bot response
        msg=self.msg_entry.get()
        self.my_msg(msg,"English")
        self.deocded_output(msg,"Decoded")

    def bagofcharacters(self,input_t):
        cv=CountVectorizer(binary=True,tokenizer=lambda txt: txt.split(),stop_words=None,analyzer='char') 
        en_in_data=[] ; pad_en=[1]+[0]*(len(self.input_characters)-1)

        cv_inp= cv.fit(self.input_characters)
        en_in_data.append(cv_inp.transform(list(input_t)).toarray().tolist())

        if len(input_t)< self.max_input_length:
          for _ in range(self.max_input_length-len(input_t)):
            en_in_data[0].append(pad_en)
    
        return np.array(en_in_data,dtype="float32")
    
    def deocded_output(self,msg,sender):
        self.text_widget.configure(state=NORMAL)
        en_in_data = self.bagofcharacters(msg.lower()+".")
        self.text_widget.insert(END,str(sender)+" : "+self.decode_sequence(en_in_data)
                                +"\n\n")
        self.text_widget.configure(state=DISABLED)
        self.text_widget.see(END)
    
    def my_msg(self,msg,sender):
        if not msg:
            return
        self.msg_entry.delete(0,END)
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END,str(sender)+" : "+str(msg)+"\n")
        self.text_widget.configure(state=DISABLED)
        

In [28]:
# run the file
if __name__=="__main__":
    LT = LangTRans()
    LT.run()


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:551: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(
